In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('data/btcusd_1-min_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7162237 entries, 0 to 7162236
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Timestamp  float64
 1   Open       float64
 2   High       float64
 3   Low        float64
 4   Close      float64
 5   Volume     float64
dtypes: float64(6)
memory usage: 327.9 MB
